In [7]:
%matplotlib inline
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from azureml.widgets import RunDetails


In [ ]:
### In Databricks:

# spark_dataframe.write \
#   .format("delta") \
#   .option("overwriteSchema", "true") \
#   .mode("overwrite") \
#   .save("/mnt/<mounted_to_blobstore>/<deltatable_name>")



In [8]:
# Check core SDK version number
import azureml.core
from azureml.core import Experiment

print("SDK version:", azureml.core.VERSION)


SDK version: 1.22.0


In [9]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep='\n')

In [10]:
# get_default_datastore returns the default datastore attached to the Workspace

# ds = ws.get_default_datastore()

# Retrieve other registered datastores
from azureml.core import Datastore, Dataset

ds = Datastore(ws, "generalpurposeaccount")

ds.name

'generalpurposeaccount'

In [11]:
delta_file = Dataset.File.from_files(path = [(ds, '')])

In [14]:
mount_context = delta_file.mount()
mount_context.start()


AzureMLException: AzureMLException:
	Message: Mount failed unexpectedly due to: Missing required package "azureml-dataset-runtime[fuse]", which can be installed by running: "/anaconda/envs/azureml_py36/bin/python" -m pip install azureml-dataset-runtime[fuse] --upgrade
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Mount failed unexpectedly due to: Missing required package \"azureml-dataset-runtime[fuse]\", which can be installed by running: \"/anaconda/envs/azureml_py36/bin/python\" -m pip install azureml-dataset-runtime[fuse] --upgrade"
    }
}

In [38]:
mount_context.mount_point

'/tmp/tmp8mznck9b'

In [39]:
import os
os.listdir(mount_context.mount_point)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 4.0.1 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyarrow<2.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).


['temp_delta', 'tweet.py']

In [13]:
from deltalake import DeltaTable

# Read the Delta Table using the Rust API
dt = DeltaTable(f"{mount_context.mount_point}/temp_delta")


In [41]:
type(dt)

deltalake.table.DeltaTable

In [42]:
dt.files()[0:5]

['part-00000-812e775d-d171-4700-a6ed-70a4118654ba-c000.snappy.parquet']

In [43]:
py_arrow_tb = dt.to_pyarrow_table()


In [44]:
pdf = py_arrow_tb.select(['home_type',
'address_area',
'SubCity',
'PostalCodeNeighborhood',
'address_neighborhood']).to_pandas(strings_to_categorical=True)

In [46]:
pdf.shape

(50000, 5)

In [47]:
pdf.head()

,home_type,address_area,SubCity,PostalCodeNeighborhood,address_neighborhood
0,commercial,Muskoka,Lake Simcoe,Lake Simcoe North Shore,Crooked Bay
1,commercial,Muskoka,Lake Simcoe,Lake Simcoe North Shore,Crooked Bay
2,commercial,Muskoka,Lake Simcoe,Lake Simcoe North Shore,Crooked Bay
3,commercial,York,Stouffville,Stouffville,Stouffville
4,commercial,York,Stouffville,Stouffville,Stouffville
